# Alarm Data Analysis

First we ASSEMBLE it, after that its been LABELED. Then its PACKAGED and these going into a CARTON. At last the Cartons get PALLETIZED   
It goes from Assembly (Harro Hoefliger) to Labeler (Krones) to Packaging (Schubert Verpacker) to Cartoner (Pester Umverpacker) to Palletizer (Pester)




|IP_TAG Name   |Machine|German|Desired Speed|Alarm File location|
|--------------|-------|------|-------------|-------------------| 
|36630901_SPEED|Assembly|Montage|32 used but 35|Y:\E00_Solostar\E6_Assembly_Line_6\E63_Montage\CSV|   
|36640901_SPEED|Labeler|Etikettierer|500|Y:\E00_Solostar\E6_Assembly_Line_6\E64_Etikettierer\AuditTrail|   
|36650901_SPEED|Packaging (Pacemaker!)|Kartonierer|450 (3pcs) or 500 (5pcs)|Y:\E00_Solostar\E6_Assembly_Line_6\E65_Kartonierer\40 - Reports|   
|36680901_SPEED|Cartoner|Endverpacker (UVP)||Y:\E00_Solostar\E6_Assembly_Line_6\E68_Endverpacker\UVP|   
|36680902_SPEED|Palletizer|Endverpacker (PAL)||Y:\E00_Solostar\E6_Assembly_Line_6\E68_Endverpacker\PAL|       



In [112]:
import pandas as pd
import numpy as np
import datetime
import os
import xlrd
import re

# this needs a settings.cfg file in the same directory
# my modules
import set_config
from common_functions import create_shift_category
from common_functions import create_df_from_file

# call set_config
dir_sanofi_share = set_config.ConfigSectionMap("SectionOne")['sanofi']
dir_local = set_config.ConfigSectionMap("SectionOne")['local']


## AL6_Endverpacker

Endverpacker folder consists of PAL and UVP Alarm files and we want to get both of those

Dir: Y:\E00_Solostar\E6_Assembly_Line_6\E68_Endverpacker\PAL    
Dir: Y:\E00_Solostar\E6_Assembly_Line_6\E68_Endverpacker\UVP

These files are converted from .VAA files into csv and then we can read them:

```

ID	State	StateID	StateText	TimeIn	TimeUserAck	TimePLCAck	MessageID	MessageText	Occur	GroupID	GroupText	Priority	Parameter1	Parameter2	Group	Class	HistoricalID	HasNotes	SortBuffer	TimeInUTC	User	Machine
579	0	$2524	MGG	31/07/2021 23:57	30/12/1899 00:00:00	01/08/2021 00:30	$9051	Maschine wartet auf Produkte	1	$8879	Warnung Maschine	0	$7326	12	42	5	1	0	Alarmhistory_1-8-21--12-0	31/07/2021 21:57		FRAM28556
579	4	$2522	MGK	01/08/2021 00:30	30/12/1899 00:00:00	30/12/1899 00:00:00	$9051	Maschine wartet auf Produkte	1	$8879	Warnung Maschine	0	$7326	12	42	5	2	0	Alarmhistory_1-8-21--12-0	31/07/2021 22:30		FRAM28556
579	0	$2524	MGG	01/08/2021 00:30	30/12/1899 00:00:00	01/08/2021 00:31	$9051	Maschine wartet auf Produkte	1	$8879	Warnung Maschine	0	$7326	12	42	5	3	0	Alarmhistory_1-8-21--12-0	31/07/2021 22:30		FRAM28556
828	4	$2522	MGK	01/08/2021 00:32	30/12/1899 00:00:00	30/12/1899 00:00:00	$9300	Roboter: Palette voll	1	$8896	Warnung Roboter: Kartonhandling	0	$8323	10004	59	5	4	0	Alarmhistory_1-8-21--12-0	31/07/2021 22:32		FRAM28556

```

In [113]:
# PAL alarms

folder = 'Alarms_data'
subfolder = 'AL6_Endverpacker'
subsubfolder = 'PAL'
path = os.path.join(dir_sanofi_share, folder, subfolder, subsubfolder)

Files = []
df = []
PAL_df = pd.DataFrame()


for filename in os.listdir( path ):
    if filename.endswith('.csv'):
        Files.append(filename)

Files.sort(key=str.lower)

# Loop through all the files.
for filename in Files:
    filepath = os.path.join(path, filename)
    # print (path)
    df = pd.read_csv(filepath, encoding='mbcs')
    df['Filename'] = filename
    PAL_df = PAL_df.append(df)


# C:\Users\mark_\Sanofi\Sanofi x McLaren sharing - General\Frankfurt sprint\SFD\Alarms_data\AL6_Endverpacker
PAL_df['Machine'] = subsubfolder

In [114]:
# UVP alarms

folder = 'Alarms_data'
subfolder = 'AL6_Endverpacker'
subsubfolder = 'UVP'
path = os.path.join(dir_sanofi_share, folder, subfolder, subsubfolder)

Files = []
df = []
UVP_df = pd.DataFrame()


for filename in os.listdir( path ):
    if filename.endswith('.csv'):
        Files.append(filename)

Files.sort(key=str.lower)

# Loop through all the files.
for filename in Files:
    filepath = os.path.join(path, filename)
    # print (path)
    df = pd.read_csv(filepath, encoding='mbcs')
    df['Filename'] = filename
    UVP_df = UVP_df.append(df)


# C:\Users\mark_\Sanofi\Sanofi x McLaren sharing - General\Frankfurt sprint\SFD\Alarms_data\AL6_Endverpacker
UVP_df['Machine'] = subsubfolder

## AL6_Etikettierer

Dir: Y:\E00_Solostar\E6_Assembly_Line_6\E64_Etikettierer\AuditTrail

There are PDF files and AuditTrail csv files which hold the same information.
- status 0 = start of alarm
- status 3 = end of alarm

Problem is the ID of the alarm is reused repeatedly and I can see multiple Status 3 returned for same ID without a Status 0 apparently initiating the alarm

csv files looks like this:

```
Nummer;Projekt;Status;Prio;Quali;Datum;Zeit;UTC Versatz;Meldetext;ID;BMK;Parameterwert;Einheit;Alt/Neu-Wert;Benutzer;Auftrag;Charge
00000001;BAS;0;4;L;15.08.2021;22:00:00;+02:00;01-01 Solostar PEN;;TYPE;0001;;N;Bediener;80758594;1F7908A

```

Approach:
- read all the xls files in from folder   
- derive Start


In [115]:
Files = []
df = []
Etikettierer_df = pd.DataFrame()

folder = 'Alarms_data'
subfolder = 'AL6_Etikettierer'
path = os.path.join(dir_sanofi_share, folder, subfolder)

for filename in os.listdir( path ):
    if filename.endswith('.csv'):
        Files.append(filename)

Files.sort(key=str.lower)

# Loop through all the files.
for filename in Files:
    path = os.path.join( dir_sanofi_share, folder, subfolder, filename)
    # print (path)
    df = pd.read_csv(path, sep=';', encoding='utf-16')
    df['Filename'] = filename    
    Etikettierer_df = Etikettierer_df.append(df)

Etikettierer_df['Machine'] = subfolder

# convert dates to datetime format
Etikettierer_df['Start'] = pd.to_datetime(Etikettierer_df['Datum'] + "."+ Etikettierer_df['Zeit'], format='%d.%m.%Y.%H:%M:%S')

# drop the rows with NaN values in ID - thought it was causing problems when trying to merge on ID, can't calc duration for them anyway, and I don't think they are relevant alarms
Etikettierer_df = Etikettierer_df[Etikettierer_df['ID'].notnull()]

# This was attempting to find a corresponding Status 3 for an ID to calculate 

# take the rows with status = 3
# Etikettierer_end = Etikettierer_df[['ID','Start']][Etikettierer_df['Status'] == 3]
# Etikettierer_end.rename(columns={'Start': 'End'}, inplace=True)

# get rid of the rows with status 3 from main df
# Etikettierer_df = Etikettierer_df[Etikettierer_df['Status'] != 3]

# merge 
# Etikettierer_df.merge(Etikettierer_end, on='ID', how='inner')

In [116]:
temp_df = Etikettierer_df[Etikettierer_df['ID'] == 364]
temp_df.set_index('Start', inplace=True)

temp_df.sort_index()
temp_df.loc['2021-06-04']

,Nummer,Projekt,Status,Prio,Quali,Datum,Zeit,UTC Versatz,Meldetext,ID,BMK,Parameterwert,Einheit,Alt/Neu-Wert,Benutzer,Auftrag,Charge,Filename,Machine
Start,,,,,,,,,,,,,,,,,,,
2021-06-04 00:11:29,140,MMA,0,3,L,04.06.2021,00:11:29,+02:00,Devicesperre wurde geschlossen durch: Transpor...,364.0,NaN,NaN,NaN,NaN,Bediener,80752924,1F051A,AL6Audit Trail 000000 2021-06-04 K747B14.csv,AL6_Etikettierer
2021-06-04 00:14:32,142,MMA,3,3,L,04.06.2021,00:14:32,+02:00,Devicesperre wurde geschlossen durch: Transpor...,364.0,NaN,NaN,NaN,NaN,Bediener,80752924,1F051A,AL6Audit Trail 000000 2021-06-04 K747B14.csv,AL6_Etikettierer
2021-06-04 00:16:50,145,MMA,0,3,L,04.06.2021,00:16:50,+02:00,Devicesperre wurde geschlossen durch: Transpor...,364.0,NaN,NaN,NaN,NaN,Bediener,80752924,1F051A,AL6Audit Trail 000000 2021-06-04 K747B14.csv,AL6_Etikettierer
2021-06-04 00:17:05,146,MMA,3,3,L,04.06.2021,00:17:05,+02:00,Devicesperre wurde geschlossen durch: Transpor...,364.0,NaN,NaN,NaN,NaN,Bediener,80752924,1F051A,AL6Audit Trail 000000 2021-06-04 K747B14.csv,AL6_Etikettierer
2021-06-04 00:18:17,151,MMA,0,3,L,04.06.2021,00:18:17,+02:00,Devicesperre wurde geschlossen durch: Transpor...,364.0,NaN,NaN,NaN,NaN,Bediener,80752924,1F051A,AL6Audit Trail 000000 2021-06-04 K747B14.csv,AL6_Etikettierer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-04 21:33:45,139,MMA,0,3,L,04.06.2021,21:33:45,+02:00,Devicesperre wurde geschlossen durch: Transpor...,364.0,NaN,NaN,NaN,NaN,Bediener,80752924,1F051A,AL6Audit Trail 210000 2021-06-04 K747B14.csv,AL6_Etikettierer
2021-06-04 21:33:48,142,MMA,3,3,L,04.06.2021,21:33:48,+02:00,Devicesperre wurde geschlossen durch: Transpor...,364.0,NaN,NaN,NaN,NaN,Bediener,80752924,1F051A,AL6Audit Trail 210000 2021-06-04 K747B14.csv,AL6_Etikettierer
2021-06-04 21:33:51,144,MMA,0,3,L,04.06.2021,21:33:51,+02:00,Devicesperre wurde geschlossen durch: Transpor...,364.0,NaN,NaN,NaN,NaN,Bediener,80752924,1F051A,AL6Audit Trail 210000 2021-06-04 K747B14.csv,AL6_Etikettierer


## AL6_Montage

Dir: Y:\E00_Solostar\E6_Assembly_Line_6\E63_Montage\CSV

- Saved as Text files but standard 'csv' files separated by semi-colon   
- Selecting just the files that begin with 'A' as they appear to be the warnings and alarms.    
- They don't have a header record   
- Don't know what the 'C*' files are   

``` 

Warnung;2021-08-05 02:00:09;2021-08-05 02:00:10;1342;1091-1B14 Standby: Teil nicht auf Abholposition Spur 14;0..Flt[1342];
Warnung;2021-08-05 02:00:09;2021-08-05 02:00:10;1343;1091-1B15 Standby: Teil nicht auf Abholposition Spur 15;0..Flt[1343];
Warnung;2021-08-05 02:00:23;2021-08-05 02:03:48;1198;1462-7B1 Standby: Max. Stau Abführband erreicht;0..Flt[1198];
Warnung;2021-08-05 02:04:20;2021-08-05 02:04:33;1198;1462-7B1 Standby: Max. Stau Abführband erreicht;0..Flt[1198];

```

In [117]:
subfolder = 'AL6_Montage'

Files = []
df = []
Montage_df = pd.DataFrame()
path = os.path.join(dir_sanofi_share, folder, subfolder)

for filename in os.listdir( path ):
    if filename.startswith('A'):
        path = os.path.join( dir_sanofi_share, folder, subfolder, filename )
        if os.path.getsize( path ) > 0:
            Files.append(filename)

Files.sort(key=str.lower)

# Loop through all the files.
for filename in Files:
    path = os.path.join(dir_sanofi_share, folder, subfolder, filename)
    # print (path)
    df = pd.read_csv(path, sep=';', encoding='cp1252', header=None)
    df['Filename'] = filename
    Montage_df = Montage_df.append(df)

Montage_df.columns=['Type','Start','End','Duration','Message Text','Some Code','Not sure','Filename']
Montage_df['Machine'] = subfolder

## AL6_Kartonierer

Dir: Y:\E00_Solostar\E6_Assembly_Line_6\E65_Kartonierer\40 - Reports

- Saved as PDF and xlsx files in a report format - warnings and alarms written to sections, if they occurred in that reporting period   
- Selecting just the xlsx files to work with.    
- interate over each file, find 'Alarme' and append row to dataframe until we find next blank row


``` 

					
Alarme					
Meldungstext	Beginn	Ende			
Waage: Ausdrucke können nicht mehr lokal gespeichert werden. Bitte legen Sie ein Speichermedium ein	"10:31:09
28.08.2021"	"11:24:19
28.08.2021"			
 =A+52-AP-U1# AS-i Fehler	"11:22:47
28.08.2021"	"11:23:22
28.08.2021"			
Druckluft überprüfen (Druck zu gering) (=A+05-SP48)	"11:23:28
28.08.2021"	"11:23:40
28.08.2021"			


In [118]:
subfolder = 'AL6_Kartonierer'

Files = []
df = []
Kartonierer_df = pd.DataFrame()

# Loop through all the files.

path = os.path.join (dir_sanofi_share, folder, subfolder)
for filename in os.listdir( path ):

    path = os.path.join(dir_sanofi_share, folder, subfolder, filename)
    df = pd.read_excel(path, na_filter=False)
    # df.set_index('Unnamed: 0', inplace=True)


    df['Filename'] = filename

    # Kartonierer_alarms = pd.DataFrame()

    for i, row in df.iterrows():
        str_temp = str(row[0])
        if (str_temp.find('Alarme') != -1): 
            alarm = True
        if row[0] == '': 
            alarm = False
        if alarm:
            Kartonierer_df = Kartonierer_df.append({'Message Text': row[0],
                                                    'Start': row[1],
                                                    'End': row[2],
                                                    'Filename': row['Filename']}, ignore_index=True)


Kartonierer_df = Kartonierer_df[Kartonierer_df['Message Text'].str.contains('Alarme|Meldu') == False]
# Kartonierer_alarms['Start'].replace('\n',' ', inplace=True)
Kartonierer_df['Start'] = Kartonierer_df['Start'].str[9:21] + ' ' + Kartonierer_df['Start'].str[0:8]
Kartonierer_df['End'] = Kartonierer_df['End'].str[9:21] + ' ' + Kartonierer_df['End'].str[0:8]

Kartonierer_df['Machine'] = subfolder
Kartonierer_df.head()


,End,Filename,Message Text,Start,Machine
2,30.03.2021 13:44:44,2691_20210401_114604_AutomaticBatchFinalReport...,Seidenader nicht bereit,30.03.2021 13:44:15,AL6_Kartonierer
3,30.03.2021 13:47:26,2691_20210401_114604_AutomaticBatchFinalReport...,"F4_403# Aufnahmefehler (=E+54-29SV1, =E+54-29SV3)",30.03.2021 13:47:15,AL6_Kartonierer
4,30.03.2021 14:03:54,2691_20210401_114604_AutomaticBatchFinalReport...,Druckluft überprüfen (Druck zu gering) (=A+05-...,30.03.2021 14:03:48,AL6_Kartonierer
5,30.03.2021 14:05:44,2691_20210401_114604_AutomaticBatchFinalReport...,Druckluft überprüfen (Druck zu gering) (=A+05-...,30.03.2021 14:05:33,AL6_Kartonierer
6,30.03.2021 14:08:16,2691_20210401_114604_AutomaticBatchFinalReport...,Druckluft überprüfen (Druck zu gering) (=A+05-...,30.03.2021 14:08:10,AL6_Kartonierer


In [119]:
Kartonierer_df.shape

(77690, 5)

In [120]:
UVP_df

,ID,State,StateID,StateText,TimeIn,TimeUserAck,TimePLCAck,MessageID,MessageText,Occur,...,Parameter2,Group,Class,HistoricalID,HasNotes,SortBuffer,TimeInUTC,User,Machine,Filename
0,248,0,$2524,MGG,30/09/2021 23:59:21,30/12/1899 00:00:00,01/10/2021 00:31:45,$7652,Maschine wartet auf Produkte,1,...,12,28,5,1,0,Alarmhistory_1-10-21--12-0,30/09/2021 21:59:21,NaN,UVP,Alarmhistory_1-10-21--12-0#VAA.csv
1,248,4,$2522,MGK,01/10/2021 00:32:22,30/12/1899 00:00:00,30/12/1899 00:00:00,$7652,Maschine wartet auf Produkte,1,...,12,28,5,2,0,Alarmhistory_1-10-21--12-0,30/09/2021 22:32:22,NaN,UVP,Alarmhistory_1-10-21--12-0#VAA.csv
2,248,0,$2524,MGG,01/10/2021 00:32:22,30/12/1899 00:00:00,01/10/2021 00:32:27,$7652,Maschine wartet auf Produkte,1,...,12,28,5,3,0,Alarmhistory_1-10-21--12-0,30/09/2021 22:32:22,NaN,UVP,Alarmhistory_1-10-21--12-0#VAA.csv
3,248,4,$2522,MGK,01/10/2021 00:33:04,30/12/1899 00:00:00,30/12/1899 00:00:00,$7652,Maschine wartet auf Produkte,1,...,12,28,5,4,0,Alarmhistory_1-10-21--12-0,30/09/2021 22:33:04,NaN,UVP,Alarmhistory_1-10-21--12-0#VAA.csv
4,248,0,$2524,MGG,01/10/2021 00:33:04,30/12/1899 00:00:00,01/10/2021 00:33:08,$7652,Maschine wartet auf Produkte,1,...,12,28,5,5,0,Alarmhistory_1-10-21--12-0,30/09/2021 22:33:04,NaN,UVP,Alarmhistory_1-10-21--12-0#VAA.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,248,4,$2522,MGK,09/09/2021 23:52:22,30/12/1899 00:00:00,30/12/1899 00:00:00,$7652,Maschine wartet auf Produkte,1,...,12,28,5,1352,0,Alarmhistory_9-9-21--12-0,09/09/2021 21:52:22,NaN,UVP,Alarmhistory_9-9-21--12-0#VAA.csv
1352,248,0,$2524,MGG,09/09/2021 23:52:22,30/12/1899 00:00:00,09/09/2021 23:52:29,$7652,Maschine wartet auf Produkte,1,...,12,28,5,1353,0,Alarmhistory_9-9-21--12-0,09/09/2021 21:52:22,NaN,UVP,Alarmhistory_9-9-21--12-0#VAA.csv
1353,248,4,$2522,MGK,09/09/2021 23:52:54,30/12/1899 00:00:00,30/12/1899 00:00:00,$7652,Maschine wartet auf Produkte,1,...,12,28,5,1354,0,Alarmhistory_9-9-21--12-0,09/09/2021 21:52:54,NaN,UVP,Alarmhistory_9-9-21--12-0#VAA.csv
1354,248,0,$2524,MGG,09/09/2021 23:52:54,30/12/1899 00:00:00,09/09/2021 23:54:57,$7652,Maschine wartet auf Produkte,1,...,12,28,5,1355,0,Alarmhistory_9-9-21--12-0,09/09/2021 21:52:54,NaN,UVP,Alarmhistory_9-9-21--12-0#VAA.csv


In [121]:
from datetime import datetime

#Calculate duration in seconds and add as a column to Endverpacker PAL and UVP. Rows without a duration are recorded as 'N/A'
# PAL
a=pd.to_datetime(PAL_df['TimePLCAck'], dayfirst=True)
b=pd.to_datetime(PAL_df['TimeIn'], dayfirst=True)
c=a-b
d=c.dt.total_seconds()
e=d.where(d >0, 'N/A')
PAL_df['Duration']=e

#Convert start time columns to datetime format
PAL_df['TimeIn']=pd.to_datetime(PAL_df['TimeIn'], dayfirst = True)
#Give a universal column name for Start Time (Start)
PAL_df=PAL_df.rename(columns={"TimeIn" : "Start"})


#Etikettierer does not include durations. A column has still been created for merging with 'N/Avalues.
Etikettierer_df['Duration']='N/A'

#Calculate duration in seconds and add as a column to Montage. Rows without a duration are recorded as 'N/A'
a=pd.to_datetime(Montage_df['End'])
b=pd.to_datetime(Montage_df['Start'])
c=a-b
d=c.dt.total_seconds()
e=d.where(d >0, 'N/A')
Montage_df['Duration']=e


# UVP
a=pd.to_datetime(UVP_df['TimePLCAck'], dayfirst=True)
b=pd.to_datetime(UVP_df['TimeIn'], dayfirst=True)
c=a-b
d=c.dt.total_seconds()
e=d.where(d >0, 'N/A')
UVP_df['Duration']=e

#Convert start time columns to datetime format
UVP_df['TimeIn']=pd.to_datetime(UVP_df['TimeIn'], dayfirst = True)
#Give a universal column name for Start Time (Start)
UVP_df=UVP_df.rename(columns={"TimeIn" : "Start"})


#Etikettierer does not include durations. A column has still been created for merging with 'N/Avalues.
# Etikettierer_df['Duration']='N/A'

#Calculate duration in seconds and add as a column to Montage. Rows without a duration are recorded as 'N/A'
a=pd.to_datetime(Montage_df['End'])
b=pd.to_datetime(Montage_df['Start'])
c=a-b
d=c.dt.total_seconds()
e=d.where(d >0, 'N/A')
Montage_df['Duration']=e



f=Etikettierer_df['Datum'] + "."+ Etikettierer_df['Zeit']
g=pd.to_datetime(f,format='%d.%m.%Y.%H:%M:%S')
Etikettierer_df['Start']=g

Montage_df['Start']=pd.to_datetime(Montage_df['Start'])


#Etikettierer_df=Etikettierer_df.rename(columns={"Zeit" : "Start"})

Kartonierer_df['Start'] = pd.to_datetime(Kartonierer_df['Start'], dayfirst=True)
# some end dates are missing so avoid the error with errors='coerce'
Kartonierer_df['End'] = pd.to_datetime(Kartonierer_df['End'], dayfirst=True, errors='coerce')
c = Kartonierer_df['End'] - Kartonierer_df['Start']
d = c.dt.total_seconds()
Kartonierer_df['Duration'] = d


In [122]:
#Create a universal column name for Message Text

PAL_df=PAL_df.rename(columns={"MessageText" : "Message Text"})
UVP_df=UVP_df.rename(columns={"MessageText" : "Message Text"})
Etikettierer_df=Etikettierer_df.rename(columns={"Meldetext" : "Message Text"})


In [123]:
#Create and populate master alarms database with useful data read across

Alarms_df=pd.DataFrame(columns=['Message Text','Start','Duration','Filename','Machine'])
Alarms_df=pd.concat([Alarms_df, Etikettierer_df, PAL_df, UVP_df, Montage_df, Kartonierer_df], join="inner")
Alarms_df=Alarms_df.reset_index()


In [124]:
Etikettierer_df[['Message Text','Start','Duration','Filename','Machine']]

,Message Text,Start,Duration,Filename,Machine
1,Devicesperre wurde geschlossen durch: Transpor...,2021-01-05 23:00:00,N/A,AL6Audit Trail 000000 2021-01-06 K747B14.csv,AL6_Etikettierer
2,Automatischer Wiederanlauf der Maschine ist aktiv,2021-01-05 23:00:06,N/A,AL6Audit Trail 000000 2021-01-06 K747B14.csv,AL6_Etikettierer
3,GMP: PC ist nicht bereit: Kamera PC 1,2021-01-05 23:00:05,N/A,AL6Audit Trail 000000 2021-01-06 K747B14.csv,AL6_Etikettierer
4,Devicestau im Auslauf,2021-01-05 23:00:20,N/A,AL6Audit Trail 000000 2021-01-06 K747B14.csv,AL6_Etikettierer
5,GMP: PC ist nicht bereit: Kamera PC 1,2021-01-05 23:01:05,N/A,AL6Audit Trail 000000 2021-01-06 K747B14.csv,AL6_Etikettierer
...,...,...,...,...,...
31,GMP: Schutzbereich wieder geschlossen - quitti...,2021-04-06 21:51:58,N/A,AL6Audit Trail 235025 2021-04-06 K747B14.csv,AL6_Etikettierer
32,Devicesperre wurde geschlossen durch: Transpor...,2021-04-06 21:52:42,N/A,AL6Audit Trail 235025 2021-04-06 K747B14.csv,AL6_Etikettierer
33,GMP: NOT-HALT ist gedrückt: NOT-HALT am Komman...,2021-04-06 21:53:31,N/A,AL6Audit Trail 235025 2021-04-06 K747B14.csv,AL6_Etikettierer
34,GMP: Der NOT-HALT muss quittiert werden,2021-04-06 21:53:31,N/A,AL6Audit Trail 235025 2021-04-06 K747B14.csv,AL6_Etikettierer


In [125]:
# don't work for large calls - so ignore for the time-being

# Messages=Alarms_df['Message Text'].unique()

# import googletrans
# from googletrans import Translator

# #Initialise
# translator = Translator(service_urls=['translate.googleapis.com'])



In [126]:
#This theoertically works, but in practice googletranslate is unreliable and it does not translate consistently
#translations = {}
#for element in Messages:
#    # add translation to the dictionary
#    translations[element] = translator.translate(element).text
    
#Alarms_df['Message Text']=Alarms_df['Message Text'].replace(translations)
#Alarms_df.to_csv('alarms.csv')

In [127]:
#Instead a list of unique alarm messages has been extracted, translated and saved as a CSV file.
filename = 'translations.csv'
folder='IP21'
path = os.path.join(dir_local, folder, filename)
df = pd.read_csv(path ,encoding='UTF-8', header=None, index_col=0)
# df = pd.read_csv(r'C:\Users\Raza-PC\Documents\McLaren\Sanofi\Sanofi\translations.csv',encoding='UTF-8',header=None,index_col=0)
translation=df.to_dict()
# Alarms_df['Message Text'] = Alarms_df['Message Text'].replace(translation[1])
Alarms_df['Message Text (English)'] = Alarms_df['Message Text'].replace(translation[1])


In [128]:
Alarms_df['Message Text (English)'][Alarms_df['Machine'] == 'AL6_Kartonierer']

1725707                                 Seidenader not ready
1725708    F4_403 # recording error (= E + 54-29SV1, = E ...
1725709    Check compressed air (pressure too low) (= A +...
1725710    Check compressed air (pressure too low) (= A +...
1725711    Check compressed air (pressure too low) (= A +...
                                 ...                        
1803392    [Q] F2_206 # Recording error booklet B gripper...
1803393    [Q] F2_206 # Recording error booklet B gripper...
1803394    [Q] F2_206 # Recording error booklet B gripper...
1803395    [Q] VT_608 # Sensor error folding box adhesive...
1803396    [Q] F2_208 # Folding box 1.1 not correctly fil...
Name: Message Text (English), Length: 77690, dtype: object

In [129]:
# this gets rid of (drops) the 'index' col and moves the newly created 'Message Text (English)' to the first col
cols = Alarms_df.columns.tolist()
cols = cols[-1:] + cols[1:-1]
Alarms_df = Alarms_df[cols]

In [130]:
folder = 'Alarms_data'
filename = 'significant_alarm_messages.xlsx'
path = os.path.join(dir_sanofi_share, folder, filename)

temp_df = pd.DataFrame()
Significant_alarms = pd.DataFrame()

for sheet_name in ['Etikettierer','Endverpacker','Montage','Kartonierer']:

    temp_df = pd.read_excel(path, sheet_name = sheet_name)
    Significant_alarms = Significant_alarms.append(temp_df)

In [131]:
Significant_alarms.rename(columns={'Alarm Message (English)':'Message Text (English)'}, inplace=True)
Significant_alarms_merged = Alarms_df.merge(Significant_alarms[['Message Text (English)','Significant']], on='Message Text (English)', how='outer')

In [132]:
Alarms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1803397 entries, 0 to 1803396
Data columns (total 6 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Message Text (English)  object        
 1   Message Text            object        
 2   Start                   datetime64[ns]
 3   Duration                object        
 4   Filename                object        
 5   Machine                 object        
dtypes: datetime64[ns](1), object(5)
memory usage: 82.6+ MB


In [133]:
Significant_alarms_merged.groupby(['Machine']).count()

,Message Text (English),Message Text,Start,Duration,Filename,Significant
Machine,,,,,,
AL6_Etikettierer,378737,378737,378737,378737,378737,364539
AL6_Kartonierer,77690,77690,77690,77669,77690,57191
AL6_Montage,859262,859262,859262,859262,859262,839597
PAL,207329,207329,207329,207329,207329,206216
UVP,280379,280379,280379,280379,280379,109461


In [134]:
# write out the Significant_alarms_merged file to the sanofi share
folder='IP21'
filename = 'alarms.csv'
path = os.path.join(dir_local, folder, filename)
Significant_alarms_merged.to_csv(path)

In [135]:
Significant_alarms_merged[Significant_alarms_merged['Message Text (English)'].str.contains('The label roll is empty')]

,Message Text (English),Message Text,Start,Duration,Filename,Machine,Significant
338865,The label roll is empty,Die Etikettenrolle ist leer,2021-01-07 01:35:12,N/A,AL6Audit Trail 000000 2021-01-07 K747B14.csv,AL6_Etikettierer,Yes
338866,The label roll is empty,Die Etikettenrolle ist leer,2021-01-07 01:35:25,N/A,AL6Audit Trail 000000 2021-01-07 K747B14.csv,AL6_Etikettierer,Yes
338867,The label roll is empty,Die Etikettenrolle ist leer,2021-01-08 23:25:51,N/A,AL6Audit Trail 000000 2021-01-09 K747B14.csv,AL6_Etikettierer,Yes
338868,The label roll is empty,Die Etikettenrolle ist leer,2021-01-08 23:26:35,N/A,AL6Audit Trail 000000 2021-01-09 K747B14.csv,AL6_Etikettierer,Yes
338869,The label roll is empty,Die Etikettenrolle ist leer,2021-01-09 00:14:04,N/A,AL6Audit Trail 000000 2021-01-09 K747B14.csv,AL6_Etikettierer,Yes
...,...,...,...,...,...,...,...
340574,The label roll is empty,Die Etikettenrolle ist leer,2021-06-24 19:18:24,N/A,AL6Audit Trail 210001 2021-06-24 K747B14.csv,AL6_Etikettierer,Yes
340575,The label roll is empty,Die Etikettenrolle ist leer,2021-09-07 19:53:23,N/A,AL6Audit Trail 210001 2021-09-07 K747B14.csv,AL6_Etikettierer,Yes
340576,The label roll is empty,Die Etikettenrolle ist leer,2021-09-07 19:59:34,N/A,AL6Audit Trail 210001 2021-09-07 K747B14.csv,AL6_Etikettierer,Yes
340577,The label roll is empty,Die Etikettenrolle ist leer,2021-04-28 19:20:54,N/A,AL6Audit Trail 211922 2021-04-28 K747B14.csv,AL6_Etikettierer,Yes


,Message Text (English),Message Text,Filename,Machine,Significant
Start,,,,,
2021-01-05 23:00:00,Device lock was closed by: conveyor belts,Devicesperre wurde geschlossen durch: Transpor...,AL6Audit Trail 000000 2021-01-06 K747B14.csv,Alarms_data,No
2021-01-05 23:01:05,Device lock was closed by: conveyor belts,Devicesperre wurde geschlossen durch: Transpor...,AL6Audit Trail 000000 2021-01-06 K747B14.csv,Alarms_data,No
2021-01-05 23:18:35,Device lock was closed by: conveyor belts,Devicesperre wurde geschlossen durch: Transpor...,AL6Audit Trail 000000 2021-01-06 K747B14.csv,Alarms_data,No
2021-01-05 23:19:25,Device lock was closed by: conveyor belts,Devicesperre wurde geschlossen durch: Transpor...,AL6Audit Trail 000000 2021-01-06 K747B14.csv,Alarms_data,No
2021-01-05 23:20:43,Device lock was closed by: conveyor belts,Devicesperre wurde geschlossen durch: Transpor...,AL6Audit Trail 000000 2021-01-06 K747B14.csv,Alarms_data,No
...,...,...,...,...,...
NaT,TM_951 # filling position 2 device F4_402: emp...,NaN,NaN,NaN,Yes
NaT,F2_224 # is started / fault,NaN,NaN,NaN,No
NaT,Balance: Printouts can no longer be saved loca...,NaN,NaN,NaN,No


,Message Text (English),Message Text,Filename,Machine,Significant
Start,,,,,
2021-01-05 23:00:00,Device lock was closed by: conveyor belts,Devicesperre wurde geschlossen durch: Transpor...,AL6Audit Trail 000000 2021-01-06 K747B14.csv,Alarms_data,No
2021-01-05 23:01:05,Device lock was closed by: conveyor belts,Devicesperre wurde geschlossen durch: Transpor...,AL6Audit Trail 000000 2021-01-06 K747B14.csv,Alarms_data,No
2021-01-05 23:18:35,Device lock was closed by: conveyor belts,Devicesperre wurde geschlossen durch: Transpor...,AL6Audit Trail 000000 2021-01-06 K747B14.csv,Alarms_data,No
2021-01-05 23:19:25,Device lock was closed by: conveyor belts,Devicesperre wurde geschlossen durch: Transpor...,AL6Audit Trail 000000 2021-01-06 K747B14.csv,Alarms_data,No
2021-01-05 23:20:43,Device lock was closed by: conveyor belts,Devicesperre wurde geschlossen durch: Transpor...,AL6Audit Trail 000000 2021-01-06 K747B14.csv,Alarms_data,No
...,...,...,...,...,...
NaT,TM_951 # filling position 2 device F4_402: emp...,NaN,NaN,NaN,Yes
NaT,F2_224 # is started / fault,NaN,NaN,NaN,No
NaT,Balance: Printouts can no longer be saved loca...,NaN,NaN,NaN,No
